# 超参数优化

神经网络的设计和训练仍然是具有挑战性和不可预测的过程。根据研究人员的知识和经验，调整这些模型的难度使训练和复现结果更像一门艺术而不是科学。造成这种困难的原因之一是机器学习模型的训练过程包括多个超参数，这些参数会影响模型在训练过程中如何拟合数据。这些超参数直接控制训练算法的行为，并对所得机器学习模型的性能产生重大影响。与内部模型参数不同（例如神经网络的权重，可以在模型训练阶段从数据中学习），超参数要在学习过程之前进行设置。

用于超参数优化的最广泛使用的方法是手动调整，这需要专业知识和专家经验。但是，在许多情况下，训练神经网络的经验还不够，研究人员诉诸于蛮力的网格搜索。此外，依靠过去的经验通常提供的是可行的方法，而不是最佳的超参数集。为了降低普通用户的技术门槛，近年来自动超参数优化已成为热门话题。

本文就简单记录下 Optuna -- 一个超参数优化框架，实现操作简单，准确，快速的超参数优化。

## 超参数优化挑战

传统上，超参数优化一直是人类的工作，因为它们在只能进行少量试验的方案中非常有效。如果您有一组具有丰富经验的研究人员在高度相似的数据上使用相同的模型，则这种手动优化方法有时称为“the graduate student search”或简称为“babysitting”，在计算效率上被认为是有效的。即使这样，它主要还是在要调整的超参数数量很少的情况下才是合理的。但是，它自然依赖于训练有素的体力劳动，可以达到可行但并非最佳的效果。此外，这是一个串行过程，因为这是一个反复试验的过程，研究人员需要从先前的试验中学习并调整后续试验的参数。在计算机集群和GPU处理器可以并行运行大量试验的世界中，人为操作可能会成为优化过程的瓶颈。因此，自动超参数优化领域开始引起机器学习社区的关注。在介绍一些最流行，最有效的自动优化技术之前，需要注意的是，相关人员的讨论可以使研究人员对要优化的超参数有一定程度的了解。这种洞察力对于研究人员不断改进模型和训练算法的能力至关重要。因此这一工作中的人应该在任何自动化过程中都保持一定了解，并确保优化过程对数据科学家可见。

近年来，几种自动优化方法变得越来越流行。最受欢迎的是：

- 网格搜索-在网格搜索中，我们为每个参数选择一组值，然后通过组合值的每种可能组合来形成一组试验。它实现起来简单，并行化也很简单。但是，由于值的数量随超参数的数量呈指数增长，因此它遭受了维数灾。
- 随机搜寻 —随机搜索被认为是网格搜索的一种变体，可以在相同的配置空间中从均匀密度中独立地绘制随机值，就像网格搜索所跨越的那样。随机搜索具有网格搜索的所有实用优势（简单，易于实现，琐碎的并行性），并在低维空间中以较小的效率降低为代价，而在高维搜索空间中以较大的幅度提高效率。在高维空间中，它比网格搜索更有效，因为我们旨在优化的超参数函数对某些维的变化比其他维更敏感。这使随机搜索成为与更高级的超参数优化算法进行比较的基准。
- 贝叶斯优化—贝叶斯优化框架具有几个关键要素。主要成分是概率替代模型，它由一个先验分布组成，该分布捕获了我们对未知目标函数的行为的信念。它还包括一个描述数据生成机制的观察模型。观察到目标的每个查询的输出后，更新先验以在目标函数的空间上产生更具信息量的后验分布。为了获得更高的效率，贝叶斯优化使用采集函数来权衡勘探和开发；他们的最优值位于代理模型的不确定性较大（探索）和/或模型预测较高的位置（探索）。贝叶斯优化算法然后通过最大化此类获取函数来选择下一个查询点。尽管贝叶斯优化的性能优于网格搜索和随机搜索，但以其形式，它有两个主要缺点：它不是并行算法，并且仅适用于连续的超参数，而不适用于分类的超参数。对于这两个问题，近年来已经开发出贝叶斯优化的几种变体。
- 超频带优化—尽管贝叶斯优化的目标是通过自适应方式选择超参数配置，以使其比标准基线（例如随机搜索）更高效，更快，但它需要解决从根本上挑战性的问题，即同步拟合和优化高维，非具有未知平滑度的凸函数，并且可能包含嘈杂的评估。超频带是一种通用技术，它通过做出最少的假设来避免此问题，并专注于解决如何在随机采样的超参数配置之间分配资源。Hyperband本质上是随机搜索的一种变体，它使用原则上的提前停止策略和SuccessiveHalving算法的扩展来分配资源。因此，在给定资源预算的情况下，Hyperband可以评估更多的超参数配置，并且在各种深度学习问题上的收敛速度都比贝叶斯优化快。

自动优化技术一直在不断发展，每两个月发布一次更新和改进的算法。可以探索这些较新的算法，例如结合了Hyperband算法和贝叶斯优化的[BOHB](https://arxiv.org/pdf/1807.01774.pdf)（贝叶斯优化和HyperBand），以及融合了遗传优化算法的思想的[PBT](https://arxiv.org/pdf/1711.09846.pdf)（Population-Based Training），等等。

下面就在PyTorch生态系统下利用Optuna尝试自动执行超参数搜索。

## Optuna: A hyperparameter optimization framework

Optuna 是一个开源自动超参数优化框架，专门为机器学习而设计。得益于其模块化的设计和现代优化功能的使用，它拥有一种轻量级的体系结构，该体系结构支持并行分布式优化以及对毫无希望的试验的修剪。

先定义俩名词：

- Study: optimization based on an objective function
- Trial: a single execution of the objective function

study的目标是通过多次trials找到超参数的最优解。Optuna 就是studies加速和自动化的框架。

首先，通过官方文档下的[视频](https://optuna.readthedocs.io/en/stable/tutorial/index.html)了解 Optuna的基本框架，了解其中的基本算法，初步了解一般对算法的选择，这里挑出一些重点简单记录下。

常用的算法有：

- Model-based:
    - TPE:bayesian optimization based on kernel fitting
    - GP:bayesian optimization based on Gaussian processes
    - CMA-ES:meta-heuristics algorithm for continuous space
- Other Methods:
    - Random search
    - Grid search
    - User-defined algorithm
    
选择的大致情况如下图：

![](pictures/QQ截图20210415162051.png)

默认的算法是TPE，如果不知道选择哪个，默认的就行了。

Optuna中设置了 Pruning strategy，如下图所示。

![](pictures/QQ截图20210415162405.png)

优化会基于学习曲线提前停止那些明显不行的trials。

Optuna的可视化也不错；此外，能识别出来哪些超参数更值得调整 -- Hyperparameter Importances。

Optuna基本使用代码模板如下图：

![](pictures/QQ截图20210415162902.png)

下面是一个简单的例子，来自：https://github.com/optuna/optuna/blob/master/examples/pytorch/pytorch_simple.py

In [1]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms

import optuna
from optuna.trial import TrialState

In [2]:
DEVICE = torch.device("cpu")
BATCHSIZE = 128
CLASSES = 10
DIR = os.getcwd()
EPOCHS = 10
LOG_INTERVAL = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 30
N_VALID_EXAMPLES = BATCHSIZE * 10

In [4]:
def get_mnist():
    # Load FashionMNIST dataset.
    train_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(DIR, train=True, download=True, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )
    valid_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(DIR, train=False, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )

    return train_loader, valid_loader

首先，定义模型

In [3]:
def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []

    in_features = 28 * 28
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, CLASSES))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)

然后是目标：

In [5]:
def objective(trial):

    # Generate the model.
    model = define_model(trial).to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # Get the FashionMNIST dataset.
    train_loader, valid_loader = get_mnist()

    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            # Limiting training data for faster epochs.
            if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break

            data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)

            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()

        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(valid_loader):
                # Limiting validation data.
                if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                output = model(data)
                # Get the index of the max log-probability.
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / min(len(valid_loader.dataset), N_VALID_EXAMPLES)

        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy

In [6]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=600)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2021-04-15 16:36:43,351] A new study created in memory with name: no-name-11c0a3bc-8495-43b9-85aa-360957996197


Extracting E:\Code\hydrus\5-dl-tools\FashionMNIST\raw\train-images-idx3-ubyte.gz to E:\Code\hydrus\5-dl-tools\FashionMNIST\raw


Extracting E:\Code\hydrus\5-dl-tools\FashionMNIST\raw\train-labels-idx1-ubyte.gz to E:\Code\hydrus\5-dl-tools\FashionMNIST\raw



Extracting E:\Code\hydrus\5-dl-tools\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to E:\Code\hydrus\5-dl-tools\FashionMNIST\raw


Extracting E:\Code\hydrus\5-dl-tools\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to E:\Code\hydrus\5-dl-tools\FashionMNIST\raw
Processing...


C:\Users\11445\miniconda3\envs\hydrus\lib\site-packages\torchvision\datasets\mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!



C:\Users\11445\miniconda3\envs\hydrus\lib\site-packages\torch\autograd\__init__.py:132: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  allow_unreachable=True)  # allow_unreachable flag
[I 2021-04-15 16:37:19,484] Trial 0 finished with value: 0.4859375 and parameters: {'n_layers': 1, 'n_units_l0': 86, 'dropout_l0': 0.48429827183831015, 'optimizer': 'RMSprop', 'lr': 1.0373063346856026e-05}. Best is trial 0 with value: 0.4859375.
[I 2021-04-15 16:37:26,052] Trial 1 finished with value: 0.75390625 and parameters: {'n_layers': 1, 'n_units_l0': 119, 'dropout_l0': 0.43323830510373035, 'optimizer': 'Adam', 'lr': 0.00017623483743615243}. Best is trial 1 with value: 0.75390625.
[I 2021-04-15 16:37:32,753] Trial 2 finished with value: 0.484375 and parameters: {'n_layers': 1, 'n_units_l0': 127, 'drop

[I 2021-04-15 16:38:03,768] Trial 12 pruned. 
[I 2021-04-15 16:38:04,483] Trial 13 pruned. 
[I 2021-04-15 16:38:10,782] Trial 14 finished with value: 0.778125 and parameters: {'n_layers': 1, 'n_units_l0': 32, 'dropout_l0': 0.45763240285159495, 'optimizer': 'Adam', 'lr': 0.016257768967406792}. Best is trial 14 with value: 0.778125.
[I 2021-04-15 16:38:11,563] Trial 15 pruned. 
[I 2021-04-15 16:38:12,941] Trial 16 pruned. 
[I 2021-04-15 16:38:19,635] Trial 17 finished with value: 0.79921875 and parameters: {'n_layers': 1, 'n_units_l0': 43, 'dropout_l0': 0.4277706541647851, 'optimizer': 'Adam', 'lr': 0.004923723025427377}. Best is trial 17 with value: 0.79921875.
[I 2021-04-15 16:38:26,523] Trial 18 finished with value: 0.803125 and parameters: {'n_layers': 2, 'n_units_l0': 55, 'dropout_l0': 0.4162865118908323, 'n_units_l1': 39, 'dropout_l1': 0.4129252734565883, 'optimizer': 'Adam', 'lr': 0.00377529672905905}. Best is trial 18 with value: 0.803125.
[I 2021-04-15 16:38:27,340] Trial 19 pru

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  72
  Number of complete trials:  28
Best trial:
  Value:  0.8265625
  Params: 
    n_layers: 1
    n_units_l0: 56
    dropout_l0: 0.38846973810012325
    optimizer: Adam
    lr: 0.010659780884583012


以上就是基本的用法。